In [ ]:
import pandas as pd
import math
import re
from bs4 import BeautifulSoup
import random

In [ ]:
!pip install bert-for-tf2
# !pip install sentencepiece

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_hub as hub

In [ ]:
from tensorflow.keras import layers
import bert

In [ ]:
print(tf.__version__)

2.7.0


In [ ]:
data = pd.read_excel('tweet_for_help.xlsx')

In [ ]:
data.head()

,index,tweet,help
0,0,Our NGO partner @manuvikasa is identifying tho...,0
1,1,"This man from AP, is stuck in Assam for a mont...",1
2,2,Respected chief minister\nI am Anoop Tiwari fr...,1
3,3,@Corp190_Sheetal hello mam I want food materia...,1
4,4,we distributed 4000 food kits \nfrom RSS Seva ...,0


In [ ]:
data = data[['tweet','help']]
data.head()

,tweet,help
0,Our NGO partner @manuvikasa is identifying tho...,0
1,"This man from AP, is stuck in Assam for a mont...",1
2,Respected chief minister\nI am Anoop Tiwari fr...,1
3,@Corp190_Sheetal hello mam I want food materia...,1
4,we distributed 4000 food kits \nfrom RSS Seva ...,0


In [ ]:
data['tweet'][0]

'Our NGO partner @manuvikasa is identifying those in severe need of help in Karnataka through GPS. They are also distributing food kits to these families, and saving them from destitution amid the ongoing #crisis. To support their initiative, please visit https://bit.ly/2xpPKMo\xa0https://twitter.com/EdelGive/status/1254697953878585345\xa0…'

In [ ]:
data.help.value_counts()

0    3911
1    1089
Name: help, dtype: int64

# Cleaning

In [ ]:
def clean_tweet(tweet):
    try:
        # tweet = BeautifulSoup(tweet, "lxml").get_text()
        # Removing the @
        tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
        # Removing the URL links
        tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
        # Keeping only letters
        tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
        # Removing additional whitespaces
        tweet = re.sub(r" +", ' ', tweet)
        tweet = re.sub(r"\.+", '.', tweet)
        return tweet
    except:
        return ""

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.tweet]

In [ ]:
data_clean[0]

'Our NGO partner is identifying those in severe need of help in Karnataka through GPS. They are also distributing food kits to these families and saving them from destitution amid the ongoing crisis. To support their initiative please visit '

In [ ]:
data_labels = data.help.values

In [ ]:
data_labels

array([0, 1, 1, ..., 0, 0, 0])

# Tokenization

In [ ]:

FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
tokenizer.tokenize("My name is Narayana")

['my', 'name', 'is', 'narayan', '##a']

In [ ]:
tokenizer.tokenize("I am suhail")

['i', 'am', 'su', '##hai', '##l']

In [ ]:
def encode_sentence(sentence):
  return ["[CLS]"] + tokenizer.tokenize(sentence) + ["[SEP]"]

In [ ]:
encode_sentence('Hey I am Galab')

['[CLS]', 'hey', 'i', 'am', 'gala', '##b', '[SEP]']

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [ ]:
data_inputs[0]

['[CLS]',
 'our',
 'ngo',
 'partner',
 'is',
 'identifying',
 'those',
 'in',
 'severe',
 'need',
 'of',
 'help',
 'in',
 'karnataka',
 'through',
 'gps',
 '.',
 'they',
 'are',
 'also',
 'distributing',
 'food',
 'kits',
 'to',
 'these',
 'families',
 'and',
 'saving',
 'them',
 'from',
 'des',
 '##ti',
 '##tu',
 '##tion',
 'amid',
 'the',
 'ongoing',
 'crisis',
 '.',
 'to',
 'support',
 'their',
 'initiative',
 'please',
 'visit',
 '[SEP]']

# Dataset creating

In [ ]:
import numpy as np

def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
  return np.char.not_equal( tokens, "[PAD]" ).astype(int)

def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [ ]:
pd.DataFrame([data_inputs[0],get_ids(data_inputs[0])])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
0,[CLS],our,ngo,partner,is,identifying,those,in,severe,need,of,help,in,karnataka,through,gps,.,they,are,also,distributing,food,kits,to,these,families,and,saving,them,from,des,##ti,##tu,##tion,amid,the,ongoing,crisis,.,to,support,their,initiative,please,visit,[SEP]
1,101,2256,17895,4256,2003,12151,2216,1999,5729,2342,1997,2393,1999,12092,2083,14658,1012,2027,2024,2036,20083,2833,18628,2000,2122,2945,1998,7494,2068,2013,4078,3775,8525,3508,13463,1996,7552,5325,1012,2000,2490,2037,6349,3531,3942,102


In [ ]:
out = encode_sentence('Hey I am Galab')
out.append('[PAD]')
out.append('[PAD]')
out.append('[PAD]')
print (out)
print (get_mask(out))
print (get_ids(out))
print (get_segments(out))

['[CLS]', 'hey', 'i', 'am', 'gala', '##b', '[SEP]', '[PAD]', '[PAD]', '[PAD]']
[1 1 1 1 1 1 1 0 0 0]
[101, 4931, 1045, 2572, 16122, 2497, 102, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)] for i,sent in enumerate(data_inputs) ]
random.shuffle(data_with_len)
data_with_len.sort(key= lambda x: x[2] )
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
                sent_lab[1]) for sent_lab in data_with_len if sent_lab[2]>7 ]

In [ ]:
len(sorted_all[len(sorted_all) -1][0][0])

130

In [ ]:

all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
next(iter(all_dataset))

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[  101,  2214,  2739,  1012, 21887,  2165,  2058,   102],
        [    1,     1,     1,     1,     1,     1,     1,     1],
        [    0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)

In [ ]:
BATCH_SIZE = 32

all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
print('Total batches = ' + str(NB_BATCHES))
NB_BATCHES_TEST = NB_BATCHES // 70
print(NB_BATCHES_TEST)
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)
next(iter(train_dataset))

Total batches = 156
2


(<tf.Tensor: shape=(32, 3, 15), dtype=int32, numpy=
 array([[[  101,  2057,  2031, ...,   102,     0,     0],
         [    1,     1,     1, ...,     1,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2054,  2023, ...,   102,     0,     0],
         [    1,     1,     1, ...,     1,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101, 22889,  2213, ...,   102,     0,     0],
         [    1,     1,     1, ...,     1,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        ...,
 
        [[  101,  1045,  2064, ..., 26419,  2232,   102],
         [    1,     1,     1, ...,     1,     1,     1],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101, 21887,  2005, ...,  1029,  1029,   102],
         [    1,     1,     1, ...,     1,     1,     1],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  1045,  1049, ...,  2393,  1012

# Building CNN model

In [ ]:
class CNN(tf.keras.Model):

  def __init__(self,
               nb_filters=50,
               FFN_units=512,
               nb_classes=2,
               dropout_rate=0.4,
               training=False,
               name='cnn'):
    
    super(CNN, self).__init__(name=name)

    self.bert_layer = hub.KerasLayer(
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1',
        trainable=False
    )
    
    self.conv1  = layers.Conv1D(filters=nb_filters,
                                 kernel_size=2,
                                 padding='valid',
                                 activation='relu')
    self.conv2  = layers.Conv1D(filters=nb_filters,
                                 kernel_size=3,
                                 padding='valid',
                                 activation='relu')
    self.conv3  = layers.Conv1D(filters=nb_filters,
                                 kernel_size=4,
                                 padding='valid',
                                 activation='relu')
    
    self.pool = layers.GlobalMaxPool1D()

    self.dense_1 = layers.Dense(units=FFN_units,
                                activation='relu')
    
    self.dropout = layers.Dropout(rate = dropout_rate)

    self.last_dense = layers.Dense(units=1,
                                    activation='sigmoid')
  
  def embed_with_bert(self, all_tokens):
    _, embs = self.bert_layer([all_tokens[:,0,:],
                                all_tokens[:,1,:],
                                all_tokens[:,2,:]])
    return embs

  def call(self,
           inputs,
           training):
    x = self.embed_with_bert(inputs)
    
    
    x_1 = self.conv1(x)
    x_1 = self.pool(x_1)

    x_2 = self.conv2(x)
    x_2 = self.pool(x_2)


    x_3 = self.conv3(x)
    x_3 = self.pool(x_3)

    merged = tf.concat([x_1, x_2, x_3], axis = 1)
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output


# Training

In [ ]:
VOCAB_SIZE = len(tokenizer.vocab)
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.4

NB_EPOCH = 5

In [ ]:
cnn = CNN(nb_filters=NB_FILTERS,
          FFN_units=FFN_UNITS,
          nb_classes=NB_CLASSES,
          dropout_rate=DROPOUT_RATE)

In [ ]:
# if NB_CLASSES == 2:
cnn.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [ ]:
cnn.fit(
        train_dataset,
        epochs=2)

Epoch 1/2
154/154 [==============================] - 33s 214ms/step - loss: 0.1688 - accuracy: 0.9381
Epoch 2/2
106/154 [===================>..........] - ETA: 8s - loss: 0.1282 - accuracy: 0.9552

# Evaluation

In [ ]:
results = cnn.evaluate(test_dataset)

2/2 [==============================] - 2s 189ms/step - loss: 0.3293 - accuracy: 0.8750


In [ ]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0) # simulates a batch

    output = cnn(inputs, training=False)


    return output

In [ ]:
get_prediction("I am stuck at abc road. I do not have any food to eat.")

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.80962]], dtype=float32)>

In [ ]:
get_prediction("I do not have any food to eat.")

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.6016399]], dtype=float32)>

In [ ]:
get_prediction("I am really happy.")

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.01664839]], dtype=float32)>

In [ ]:
get_prediction("Help me.")

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.2685689]], dtype=float32)>

In [ ]:
get_prediction("I donot have any food to eat, help me.")

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.8366738]], dtype=float32)>